<a href="https://colab.research.google.com/github/masaers/workshop-numpy/blob/main/pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import geopy.distance

## Fetch data
We will fetch data about all weather stations in Sweden that have ever reported air temperature, and put them in a Pandas data frame for future reference. By separating the raw data (`fetched_df`) from the data we are trying to process (`df`), we can go back to the raw data without having to refetch it.

In [ ]:
base_url = "https://opendata-download-metobs.smhi.se"
parameter = 1 # Air temperature
url = f"{base_url}/api/version/1.0/parameter/{parameter}/station.json"
fetched_df = pd.json_normalize(requests.get(url).json()["station"])

## What do we have?

We make a copy of the raw data and take a look at it. Both the values and their data types (`dtype`) are interesting.

In [ ]:
df = fetched_df.copy()
display(df)
display(df.dtypes)

## Redundant data

1. It looks like `key` and `id` are the same value, but in string (`dtype=object`) and `int64` form, so we should remove `key`.

2. It also looks like `summary` is some kind of text representation of the record, that we could re-generate from the other data, so we should remove it.

3. The `link` field does contain unique data, but not the kind we can process as "data" (and it can be generated by knowing the API), so we will remove that as well.

4. Finally, the `title` field is a combination of the `name` and the parameter that we requested via the API, so it does not add any meaningful information and should be removed.

In [ ]:
df = df[[c for c in df.columns if c not in ["key", "summary", "link", "title"]]]
display(df)
display(df.dtypes)

## Index

The `id`/`key` column was clearly intended as a unique identifier. Let's use it as index!

In [ ]:
df = df.set_index("id", drop=True).sort_index()
display(df)
display(df.dtypes)

## Conversions

The conversion from JSON to Python did a good job of converting the types as we got them, but the `updated`, `from` and `to` fields are actually timestamps, so let's use Pandas to interpret the numerical representation correctly.

Notice that the type changes to `datetime64[ns]`, a type that is native to Numpy :-)

In [ ]:
for c in ["updated", "from", "to"]:
    df[c] = pd.to_datetime(df[c], unit="ms")
display(df)
display(df.dtypes)

## Categories

It also looks like there is some categorical data (`owner`, `ownerCategory` and `measuringStations`). Let's go ahead and convert it to categories!

In [ ]:
for c in ["owner", "ownerCategory", "measuringStations"]:
    df[c] = df[c].astype("category")
display(df)
display(df.dtypes)

## Basic statistics

Very easy to obtain empirical distributions!

Why do we not get any standard deviation for timestamps?

In [ ]:
df.describe()

In [ ]:
df["updated"].std()

## Looking at the data

Easy to plot!

Let's start by looking at the active time span of stations.

In [ ]:
df.plot.scatter("to", "from")

Hard to tell active from recently decommissioned, let's color the dots according to the `active` column value.

In [ ]:
color = df["active"].apply(lambda active: "green" if active else "red")
df.plot.scatter("to", "from", c=color)

In [ ]:
df[df["from"] == df["from"].min()]

In [ ]:
df[df["to"] == df["to"].min()]

We can do histograms easily as well. Let's plot histograms over positions!

In [ ]:
df["height"].plot.hist()
plt.show()
df["latitude"].plot.hist(bins=100, color="green")
plt.show()
df["longitude"].plot.hist(bins=100, color="red")
plt.show()

Can we plot a "map"? Kinda!

In [ ]:
df.plot.scatter("longitude", "latitude").set_aspect(1)

## Break it down!

We can group by column values and aggregate associated values.

The `observed=True` (or `False`) is only needed to avoid a warning.

In [ ]:
df[["owner", "height", "latitude", "longitude"]].groupby("owner", observed=True).agg(["min", "mean", "max"])

Looks like some of the owners have exactly one station... let's verify that by adding a count. This requires different aggregations, so we can take this opportunity to systematize a bit.

In [ ]:
group = ["owner"]
agg = { c: ["min", "mean", "max"] for c in ["height", "latitude", "longitude"] }
agg["owner"] = "count"
display(agg)
columns = list(set(group + list(agg.keys())))
df[columns].groupby(group, observed=True).agg(agg)

We can "fix" this by clumping low frequency owners togeather as a new `OTHER` category.

In [ ]:
display(df["owner"].cat.categories)
df["owner"] = df["owner"].cat.add_categories("OTHER")
display(df["owner"].cat.categories)

I got this recipe off of StackOverflow, so I want to run each step of it to see what it does, so that I understand why it works.

In [ ]:
display(df["owner"].value_counts())
display(df["owner"].value_counts()[df["owner"]])
display(df["owner"].value_counts()[df["owner"]].values)
display(df.loc[df["owner"].value_counts()[df["owner"]].values < 5, "owner"])

In [ ]:
# From StackOverflow https://stackoverflow.com/a/22208838/1017986
df.loc[df["owner"].value_counts()[df["owner"]].values < 5, "owner"] = "OTHER"
display(df[df["owner"] == "OTHER"])

In [ ]:
group = ["owner"]
agg = { c: ["min", "mean", "max"] for c in ["height", "latitude", "longitude"] }
agg["owner"] = "count"
columns = list(set(group + list(agg.keys())))
df[columns].groupby(group, observed=True).agg(agg)

In [ ]:
df[df["height"] < 0]

In [ ]:
group = ["owner", "active", "ownerCategory"]
agg = { c: ["min", "mean", "max"] for c in ["height", "latitude", "longitude"] }
agg["owner"] = "count"
columns = list(set(group + list(agg.keys())))
df[columns].groupby(group, observed=True).agg(agg)

## Sorting

How far away are these stations from us?

In [ ]:
HIQ_STO_LATLON = (59.319268, 18.075664)
df["from_here"] = df.apply(lambda row: geopy.distance.distance(HIQ_STO_LATLON, (row["latitude"], row["longitude"])).km, axis=1)
display(df[df["active"]].sort_values(by="from_here"))

## My favorite algorithm!

My favorite algorithm is [*k*-means](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.vq.kmeans.html) clustering. Let's reimagine Sweden's 21 "regioner" based on how weather stations are distributed!

Time to break out SciPy!

In [ ]:
x = df[["longitude", "latitude"]]
print(f"{x.shape=}")
centroids, distortion = scipy.cluster.vq.kmeans(x, 21)
print(f"{centroids.shape=} {distortion=}")

## Show on "map"

SciPy also let's us construct and graph Voroni tesselations!

In [ ]:
df.plot.scatter("longitude", "latitude", color="pink").set_aspect(1)
voroni = scipy.spatial.Voronoi(centroids)
scipy.spatial.voronoi_plot_2d(voroni, plt.gca(), show_vertices=False, line_colors="red")
plt.show()

## Seeding centroids?

Can we do better by seeding *k*-means with current regions?

I found a CSV with the location of Swedish cities with Municipality (kommun?) and County (region?) as well as longitude/latitude coordinates on GitHub. Turns out you can just read a URL!

No population data, but we can still use the median city in each region as initial centroid.

In [ ]:
cities_df = pd.read_csv("https://raw.githubusercontent.com/sphrak/svenska-stader/refs/heads/master/src/svenska-stader.csv")
display(cities_df)
display(cities_df[["County", "Longitude", "Latitude"]].groupby("County").median())
display(cities_df[["County", "Longitude", "Latitude"]].groupby("County").median().values)

In [ ]:
x = df[["longitude", "latitude"]]
guess = cities_df[["County", "Longitude", "Latitude"]].groupby("County").median().values
centroids, distortion = scipy.cluster.vq.kmeans(x, guess)

In [ ]:
df.plot.scatter("longitude", "latitude", color="pink").set_aspect(1)
voroni = scipy.spatial.Voronoi(centroids)
scipy.spatial.voronoi_plot_2d(voroni, plt.gca(), show_vertices=False, line_colors="red")
plt.show()